In [1]:
%matplotlib inline


# 3D map analysis


## Context

While 1D spectral analysis is sufficient model a single point-source in the ON region, 3D analyses allow to consider complex field of views containing overlapping gamma-ray sources and model their respectibe morphologies.


## Proposed approach

In practice, we have to:

- Create a `~gammapy.data.DataStore` pointing to the relevant data
- Apply an observation selection to produce a list of observations,
  a `~gammapy.data.Observations` object.
- Define a geometry of the `~gammapy.datasets.MapDataset` we want to produce, with a sky projection
  and an energy range :
    - Create a `~gammapy.maps.MapAxis` for the energy
    - Create a `~gammapy.maps.WcsGeom` for the geometry
    
- Define the`~gammapy.modeling.models.Models` to apply to the dataset.
    - Load the known source from previous catalogues for example `gammapy.catalog.SourceCatalogHGPS`
    - Read one of the `gammapy.modeling.models.TemplateSpatialModel` provided for the diffuse emission
    - Use the known sources to create exclusion region map
  
- Create the necessary makers:

  - the map dataset maker `~gammapy.makers.MapDatasetMaker`
  - the background normalization maker, here a `~gammapy.makers.FoVBackgroundMaker` 
  - and usually the safe range maker : `~gammapy.makers.SafeMaskMaker`

- Perform the data reduction loop :
  - create a stacked `~gammapy.datasets.MapDataset` using the `~gammapy.makers.DatasetsMaskMaker`

- Make a TSmap using `~gammapy.estimators.TSMapEstimator`
- Create a `~gammapy.modeling.Fit` object and run it to fit the model
  parameters
- Use `~gammapy.estimators.utils.find_peaks_in_flux_map` to find new source candidates
- Fit, add new source models and re-fit until we get flat residual in the TSmap
- Usage of `~gammapy.modeling.select_nested_models` to compute significance for :
    - source detection
    - spectral curvature
    - spatial extension.


## Setup

First, we setup the analysis by performing required imports.


In [2]:
from pathlib import Path
from astropy import units as u
from astropy.coordinates import SkyCoord
from regions import CircleSkyRegion, Regions
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from IPython.display import display

from gammapy.catalog import CATALOG_REGISTRY
from gammapy.data import DataStore
from gammapy.datasets import MapDataset
from gammapy.estimators import TSMapEstimator
from gammapy.makers import FoVBackgroundMaker, MapDatasetMaker, SafeMaskMaker, DatasetsMaker
from gammapy.maps import MapAxis, WcsGeom
from gammapy.modeling import Fit
from gammapy.modeling.models import (
    FoVBackgroundModel,
    PointSpatialModel,
    PowerLawSpectralModel,
    LogParabolaSpectralModel,
    SkyModel,
    TemplateSpatialModel,
    PowerLawNormSpectralModel,
    Models,
    GeneralizedGaussianSpatialModel
)
from gammapy.utils.check import check_tutorials_setup
from gammapy.visualization import plot_npred_signal

from gammapy.utils import pbar
pbar.SHOW_PROGRESS_BAR = True

## Defining the datastore and selecting observations

We first use the `~gammapy.data.DataStore` object to access the
observations we want to analyse. Here we are going to use the simulated observations from CTA-1DC (first technical data challenge).


In [3]:
data_store = DataStore.from_dir("$GAMMAPY_DATA/cta-1dc/index/gps/")

We can now define an observation filter to select only the relevant
observations. Here we use a cone search which we define with a python
dict.

We then filter the `ObservationTable` with
`~gammapy.data.ObservationTable.select_observations`.




OBS_ID,RA_PNT,DEC_PNT,GLON_PNT,GLAT_PNT,ZEN_PNT,ALT_PNT,AZ_PNT,ONTIME,LIVETIME,DEADC,TSTART,TSTOP,DATE-OBS,TIME-OBS,DATE-END,TIME-END,N_TELS,OBJECT,CALDB,IRF,EVENTS_FILENAME,EVENT_COUNT
,deg,deg,deg,deg,deg,deg,deg,s,s,,s,s,,,,,,,,,,
int64,float64,float64,float64,float64,float64,float64,int64,float64,float64,float64,float64,float64,bytes10,bytes8,bytes10,bytes8,int64,bytes21,bytes3,bytes13,bytes50,int64
110380,267.68121338,-29.6075,359.9999912037958,-1.299995937905366,0.0,90.0,0,1800.0,1764.0,0.98000001907,664502400.0,664504192.0,2021-01-21,11:58:51,2021-01-21,12:28:51,0,Galactic Plane Survey,1dc,South_z20_50h,1dc/1dc/data/baseline/gps/gps_baseline_110380.fits,106217
111140,264.2315979,-29.5214,358.4999833830074,1.3000020211954284,0.0,90.0,0,1800.0,1764.0,0.98000001907,667958400.0,667960192.0,2021-03-02,11:58:51,2021-03-02,12:28:51,0,Galactic Plane Survey,1dc,South_z20_50h,1dc/1dc/data/baseline/gps/gps_baseline_111140.fits,106386
111159,266.03671265,-26.9782,1.5000056568267741,1.299940468335294,0.0,90.0,0,1800.0,1764.0,0.98000001907,668044800.0,668046592.0,2021-03-03,11:58:51,2021-03-03,12:28:51,0,Galactic Plane Survey,1dc,South_z20_50h,1dc/1dc/data/baseline/gps/gps_baseline_111159.fits,105741


We can now retrieve the relevant observations by passing their
``obs_id`` to the `~gammapy.data.DataStore.get_observations`
method.




Obs Id:   0%|          | 0/3 [00:00<?, ?it/s]

And look at their details

## Preparing reduced datasets geometry

Now we define a reference geometry for our analysis, We choose a WCS
based geometry with a binsize of 0.02 deg and also define an energy
axis:




Now we can define the target dataset with this geometry.




## Prepare the models for known sources

First we load the H.E.S.S. Galactic plane survey (HGPS) source catalog

and select the sources inside our analysis region

## Prepare the models for known sources and diffuse emission 


Now we load an intersellar emission model  (IEM) created with https://github.com/cosmicrays/hermes for the CTA-GPS simulation https://zenodo.org/records/10008527. We added lighter versions in the repo so you can direclty read one of theses using `TemplateSpatialModel.read` method:


In [11]:
path = Path("./models/diffuse_template")
template_iem_base = TemplateSpatialModel.read(path / "IEM_base_20deg.fits.gz", normalize=False) 
template_varmin_rescaled = TemplateSpatialModel.read(path / "IEM_varmin_rescaled_20deg.fits.gz", normalize=False) 
#which one is the best suited for this simulation ? we will try later


These template are 3D maps so the `normalize=False` option is important to keep spectral information unchanged. 
We add that into a `SkyModel` together with a `PowerLawNormSpectralModel` that allows to refit the normalisation of the template.

## Data reduction


### Prepare exclusion mask

In order to scale the background model we have to mask the region with significant emission as explained in https://docs.gammapy.org/1.2/tutorials/api/mask_maps.html#creating-an-exclusion-mask

We start with know sources :


We add a $\pm 1$ deg band to mask the contribution from diffuse emission in the Galactic plane

[<RectangleSkyRegion(center=<SkyCoord (Galactic): (l, b) in deg
    (0., 0.)>, width=20.0 deg, height=2.0 deg, angle=0.0 deg)>]


 to define the exclusion mask we take the inverse and plot the resulting map to make sure it is correct


### Create the maker classes to be used

The `~gammapy.makers.MapDatasetMaker` object is initialized as well as
the `~gammapy.makers.SafeMaskMaker` that carries here a maximum offset
selection and the `~gammapy.makers.FoVBackgroundMaker` which will normalize the background.

We will put them into a list to later be used with the `~gammapy.makers.DatasetsMaker`



### Perform the data reduction

Use the DatasetsMaker and run it on the empty dataset and the observation list



## Save dataset to disk

It is common to run the preparation step independent of the likelihood
fit, because often the preparation of maps, PSF and energy dispersion is
slow if you have a lot of data. We first create a folder:




In [18]:
path = Path("datasets/cta1dc_gc/")
path.mkdir(exist_ok=True)

And then write the datasets to disk by calling the dedicated
`~gammapy.datasets.Datasets.write` method:




In [19]:
filename = path / "datasets_cta1dc_gc.yaml"
datasets.write(filename, overwrite=True)